```{contents}
```
## **Agent Communication in LangGraph**

Agent communication in LangGraph is the **formal mechanism by which multiple autonomous agents exchange information, coordinate actions, negotiate decisions, and jointly solve complex tasks** using a shared, state-driven execution model.

Unlike chat-based multi-agent frameworks, LangGraph implements communication as **structured state transitions inside a persistent execution graph**.

---

### **1. Core Principle**

> **Agents do not message each other directly.
> They communicate by reading from and writing to a shared, versioned state.**

This transforms communication from **unstructured text exchange** into **deterministic dataflow**.

---

### **2. Communication Architecture**

```
Agent A ─┐
         ├──► Shared State ◄─── Agent B
Agent C ─┘
```

Each agent:

* Reads required fields from the state
* Writes structured updates
* Triggers routing decisions

---

### **3. State as Communication Bus**

```python
class State(TypedDict):
    task: str
    research: str
    solution: str
    critique: str
    final: str
```

All agents use this schema to exchange information.

| Field    | Producer       | Consumer |
| -------- | -------------- | -------- |
| task     | User           | Planner  |
| research | Research Agent | Solver   |
| solution | Solver         | Critic   |
| critique | Critic         | Solver   |
| final    | Supervisor     | Output   |

---

### **4. Agent Node Implementation**

```python
def research_agent(state):
    return {"research": f"Facts about {state['task']}"}

def solver_agent(state):
    return {"solution": f"Draft solution using {state['research']}"}

def critic_agent(state):
    return {"critique": f"Issues found in {state['solution']}"}
```

Each function **publishes its output** into the shared state.

---

### **5. Routing & Turn-Taking**

Communication order is controlled by the graph.

```python
builder.add_edge("research", "solve")
builder.add_edge("solve", "critic")
builder.add_edge("critic", "solve")   # feedback loop
```

This produces **iterative communication** between agents.

---

### **6. Negotiation & Consensus**

```python
def supervisor(state):
    if "no issues" in state["critique"]:
        return {"final": state["solution"], "done": True}
    return {"done": False}
```

```python
builder.add_conditional_edges("supervisor",
    lambda s: END if s["done"] else "solve",
    {"solve": "solve", END: END}
)
```

Agents negotiate through **state updates**, not chat.

---

### **7. Human-in-the-Loop Communication**

Humans are treated as agents:

```python
def human_review(state):
    return {"solution": input("Corrected solution: ")}
```

---

### **8. Communication Patterns**

| Pattern    | Description                        |
| ---------- | ---------------------------------- |
| Pipeline   | Sequential handoff                 |
| Debate     | Competing agent outputs            |
| Consensus  | Supervisor merges results          |
| Blackboard | All agents read/write shared state |
| ReAct      | Reason–Act–Observe loop            |
| Reflection | Critic feedback cycles             |

---

### **9. Advantages Over Message Passing**

| LangGraph     | Chat-based Agents |
| ------------- | ----------------- |
| Deterministic | Non-deterministic |
| Inspectable   | Hard to debug     |
| Versioned     | No state history  |
| Serializable  | Ephemeral         |
| Recoverable   | Not recoverable   |

---

### **10. Minimal Working Example**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    task: str
    research: str
    solution: str
    critique: str
    done: bool

builder = StateGraph(State)

builder.add_node("research", research_agent)
builder.add_node("solve", solver_agent)
builder.add_node("critic", critic_agent)
builder.add_node("supervisor", supervisor)

builder.set_entry_point("research")
builder.add_edge("research", "solve")
builder.add_edge("solve", "critic")
builder.add_edge("critic", "supervisor")

builder.add_conditional_edges("supervisor",
    lambda s: END if s["done"] else "solve",
    {"solve": "solve", END: END}
)

graph = builder.compile()
graph.invoke({"task": "Quantum computing", "done": False})
```

---

### **11. Production Considerations**

| Concern     | Solution            |
| ----------- | ------------------- |
| Conflicts   | Versioned state     |
| Deadlocks   | Routing constraints |
| Scalability | Async nodes         |
| Audit       | State snapshots     |
| Safety      | Approval gates      |

---

### **12. Mental Model**

> **LangGraph agents communicate the same way distributed systems do — through shared state, controlled scheduling, and deterministic transitions.**

This is what makes LangGraph suitable for **enterprise-grade multi-agent systems**.

### Demonstration

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

# -------------------- Shared State --------------------
class State(TypedDict):
    task: str
    research: str
    solution: str
    critique: str
    iteration: int
    done: bool

# -------------------- Agents --------------------
def research_agent(state: State):
    return {"research": f"Key facts about {state['task']}."}

def solver_agent(state: State):
    improved = state["solution"] + " Improved." if state["solution"] else "Initial draft."
    return {"solution": improved}

def critic_agent(state: State):
    if state["iteration"] >= 2:
        return {"critique": "No issues."}
    return {"critique": "Needs improvement."}

def supervisor_agent(state: State):
    next_iter = state["iteration"] + 1
    finished = "No issues." in state["critique"]
    return {"iteration": next_iter, "done": finished}

# -------------------- Graph --------------------
builder = StateGraph(State)

builder.add_node("research", research_agent)
builder.add_node("solve", solver_agent)
builder.add_node("critic", critic_agent)
builder.add_node("supervisor", supervisor_agent)

builder.set_entry_point("research")
builder.add_edge("research", "solve")
builder.add_edge("solve", "critic")
builder.add_edge("critic", "supervisor")

builder.add_conditional_edges(
    "supervisor",
    lambda s: END if s["done"] else "solve",
    {"solve": "solve", END: END}
)

graph = builder.compile()

# -------------------- Run --------------------
final_state = graph.invoke({
    "task": "Autonomous vehicles",
    "research": "",
    "solution": "",
    "critique": "",
    "iteration": 0,
    "done": False
})

final_state


{'task': 'Autonomous vehicles',
 'research': 'Key facts about Autonomous vehicles.',
 'solution': 'Initial draft. Improved. Improved.',
 'critique': 'No issues.',
 'iteration': 3,
 'done': True}